In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


### 멜론 인기 테마 / 인기 장르 
- 메인 Tag URL : https://www.melon.com/dj/tag/djtaghub_list.htm?tagSeq=43#params%5BtagSeq%5D=43&params%5BorderBy%5D=POP&po=pageObj&startIndex=1
- PlayListURL : https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=리스트ID


##### 1. 인기테마/장르 별로 플레이리스트 ID가 저장된 파일을 불러온다.

In [22]:
df = pd.read_csv('data/멜론테그번호.CSV', encoding='euc-kr')
print(df.shape)
df.head()

(44, 2)


,tag_name,id
0,가을,2
1,기분전환,3
2,휴식,4
3,힐링,5
4,사랑,6


##### 2. 해당 ID별로 리스트 목록을 가지고 와서 각각(2페이지) 50개 내의 노래 리스트만 가지고온다.

In [12]:
tag_list = df.id.values
# 메인 tag URL
tag_url1 = 'https://www.melon.com/dj/tag/djtaghub_list.htm?tagSeq='
tag_url2 = '#params%5BtagSeq%5D='
tag_url3 = '&params%5BorderBy%5D=POP&po=pageObj&startIndex='
# PlayList URL
playlist_url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq='

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
# driver = webdriver.Chrome('C:/Users/yonsai/Downloads/chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome('C:/Users/sujung/Downloads/chromedriver_win32/chromedriver.exe')


In [19]:
plist1, plist2 = [], []

for i in tqdm(tag_list):

    # 1, 2 페이지의 내용만
    for k in range(1, 22, 20):

        driver.get(f'{tag_url1}{i}{tag_url2}{i}{tag_url3}{k}')
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        lis = soup.select('.service_list_play.d_djcol_list > ul > li > div > a')
        
        plist = [li['href'].split(',')[-1][1:-3] for li in lis]

        for pli in plist:

            driver.get(playlist_url + pli)
            time.sleep(1)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # plist1에 들어갈 요소들
            title = soup.select_one('.ellipsis.song_name').text.strip()
            tag = soup.select_one('.tag_list.type03').text.replace('\n', ' ')
            like = int(soup.select_one('.cnt').text.split()[-1].replace(',', '').strip())
            
            trs = soup.select('.service_list_song.d_song_list > table > tbody > tr' )
            song_ids = []
            # plist2에 들어갈 요소들
            for tr in trs:
                
                if tr.select_one('.bullet_icons.age_19') :  # 19금 제외
                    continue

                img = tr.select_one('img')['src']

                tmp = tr.select_one('.rank01 > span > a')
                s_id = tmp['href'].split(',')[-1][:-2]
                song_ids.append(s_id)

                s_title = tmp.text.strip()
                s_artist = tr.select_one('.rank02 > span').text

                plist2.append({'plylstSeq': pli, 'songId': s_id, 'title': s_title, 'artist': s_artist, 'img': img})
            
            plist1.append({'plylstSeq': pli, 'title': title, 'tag': tag, 'like':like, 'songIds': ' '.join(song_ids)})


df1 = pd.DataFrame(plist1)
df2 = pd.DataFrame(plist2)

print(df1.shape, df2.shape)

df1.to_csv('data/playlist1.csv', index=False)
df2.to_csv('data/playlist2.csv', index=False)

driver.close()


100%|██████████| 20/20 [01:00<00:00,  3.03s/it]

(20, 5) (985, 5)


In [90]:
df2.shape[0], df2.songId.nunique()

(33922, 13806)

In [25]:
df1.head()

,plylstSeq,title,tag,like,songIds
0,436119495,밤 거리를 걸으며 듣기 좋은 노래:),#이별하고#추억#위로#잔잔한#새벽#인디#발라드#감성#기분_전환#산책해요,10860,36243156 34547880 35963069 35869352 35869172 3...
1,523770321,"[2011-2014] 그때 그 시절, 추억이 새록새록 떠오르는 노래 모음",#추억#기분전환#2010년대#드라이브#산책#힐링#감성#휴식#회상#그리움,1257,4192787 3170749 4300699 4129672 3890826 468114...
2,507012761,미소짓게 만드는 산뜻하고 기분좋은 미디엄템포 노래 (상시 업데이트),#봄#카페#인디#봄바람#매력적인#커피#사랑#미디엄템포#기분전환#기분좋아지는,5722,35802587 35333036 36206259 36266618 4295943 36...
3,520432487,몸이 먼저 반응하는 90년대 2000년대 메가 히트 댄스곡,#댄스#추억#신나는#기분전환#회상#1990년대#2000년대#드라이브#명곡#노래방,1386,71606 78184 27654 66964 191093 85258 83190 104...
4,522919220,언제 들어도 좋은 2010년대 발라드 히트곡,#발라드#감성#사랑#잔잔한#기분전환#일상#힐링#휴식#2010년대,1244,4142913 30177818 32224383 8102386 3820647 3701...


In [26]:
df2.head()

,plylstSeq,songId,title,artist,img
0,436119495,36243156,고맙소,한올,https://cdnimg.melon.co.kr/cm2/album/images/11...
1,436119495,34547880,편지,n@di (나디),https://cdnimg.melon.co.kr/cm2/album/images/10...
2,436119495,35963069,"나의 모든 이들에게,",Mingginyu (밍기뉴),https://cdnimg.melon.co.kr/cm2/album/images/11...
3,436119495,35869352,사랑이 사랑인줄 몰랐으니까,한올,https://cdnimg.melon.co.kr/cm2/album/images/11...
4,436119495,35869172,널 바래다주던 길,마인드유,https://cdnimg.melon.co.kr/cm2/album/images/11...


##### 3. PlaylistID를 알 때 크롤링

In [1]:
# 두 파일에서 PlayListID를 알아온다.
ply_list = '100123440 482975847'.split()

In [ ]:
plist1, plist2 = [], []

for ply in ply_list:
    
    driver.get(playlist_url + ply)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # plist1에 들어갈 요소들
    title = soup.select_one('.ellipsis.song_name').text.strip()
    tag = soup.select_one('.tag_list.type03').text.replace('\n', ' ')
    like = int(soup.select_one('.cnt').text.split()[-1].replace(',', '').strip())
    
    trs = soup.select('.service_list_song.d_song_list > table > tbody > tr' )
    song_ids = []
    # plist2에 들어갈 요소들
    for tr in trs:
        
        if tr.select_one('.bullet_icons.age_19') :  # 19금 제외
            continue

        img = tr.select_one('img')['src']

        tmp = tr.select_one('.rank01 > span > a')
        s_id = tmp['href'].split(',')[-1][:-2]
        song_ids.append(s_id)

        s_title = tmp.text.strip()
        s_artist = tr.select_one('.rank02 > span').text

        plist2.append({'plylstSeq': pli, 'songId': s_id, 'title': s_title, 'artist': s_artist, 'img': img})
    
    plist1.append({'plylstSeq': pli, 'title': title, 'tag': tag, 'like':like, 'songIds': ' '.join(song_ids)})